<a href="https://colab.research.google.com/github/Kzis/map_reduce/blob/master/spark_mapreduce_2gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz


In [0]:
!tar xf spark-2.4.3-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init('/content/spark-2.4.3-bin-hadoop2.7')

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

In [0]:
import csv

line = []

with open('test.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\n')
    for row in csv_reader:
        line.append(row[0])
        
print(line)

In [0]:
import collections


def mapper2gram(all_data):
  map_list = []
  
  for line in all_data:
    
    words = line.split()
    words2 = collections.deque(words)
    words2.rotate(1)
    
    words = list(map(lambda a , b: a+" "+b ,list(words2), words))
    del words[0]
    
    map_list.append(words)
  return map_list

def extract2gram(map_list):
  extract_list = []
  for comp in map_list:
    for inx in comp:
      extract_list.append(inx)
  return extract_list
      

In [50]:
map_list = mapper2gram(line)
extract_list = extract2gram(map_list)

print(map_list)
print(extract_list)

[['1 1', '1 1', '1 2', '2 2', '2 2', '2 3', '3 3', '3 3', '3 4', '4 5', '5 6'], ['a b', 'b c', 'c 1', '1 2', '2 7', '7 8', '8 9', '9 5', '5 6', '6 b', 'b c']]
['1 1', '1 1', '1 2', '2 2', '2 2', '2 3', '3 3', '3 3', '3 4', '4 5', '5 6', 'a b', 'b c', 'c 1', '1 2', '2 7', '7 8', '8 9', '9 5', '5 6', '6 b', 'b c']


In [53]:
x = sc.parallelize(extract_list)
print(x.collect())

wc = x.flatMap(lambda v: v.split(","))
print(wc.collect())

y = x.map(lambda x: ((x, 1)))
print(y.collect())

z = y.reduceByKey(lambda a, b: a + b)
print(z.collect())

['1 1', '1 1', '1 2', '2 2', '2 2', '2 3', '3 3', '3 3', '3 4', '4 5', '5 6', 'a b', 'b c', 'c 1', '1 2', '2 7', '7 8', '8 9', '9 5', '5 6', '6 b', 'b c']
['1 1', '1 1', '1 2', '2 2', '2 2', '2 3', '3 3', '3 3', '3 4', '4 5', '5 6', 'a b', 'b c', 'c 1', '1 2', '2 7', '7 8', '8 9', '9 5', '5 6', '6 b', 'b c']
[('1 1', 1), ('1 1', 1), ('1 2', 1), ('2 2', 1), ('2 2', 1), ('2 3', 1), ('3 3', 1), ('3 3', 1), ('3 4', 1), ('4 5', 1), ('5 6', 1), ('a b', 1), ('b c', 1), ('c 1', 1), ('1 2', 1), ('2 7', 1), ('7 8', 1), ('8 9', 1), ('9 5', 1), ('5 6', 1), ('6 b', 1), ('b c', 1)]
[('1 1', 2), ('1 2', 2), ('2 2', 2), ('2 3', 1), ('3 3', 2), ('3 4', 1), ('4 5', 1), ('5 6', 2), ('a b', 1), ('b c', 2), ('c 1', 1), ('2 7', 1), ('7 8', 1), ('8 9', 1), ('9 5', 1), ('6 b', 1)]
